# Bus Price Prediction - Dask-Based Full Dataset Training

**Improvements over batch training:**
1. Proper global shuffle before train/test split
2. Single model training (no incremental tree accumulation issues)
3. Consistent feature encoding across all data
4. Memory-efficient processing with Dask

**Expected R²: ~0.70+ (similar to small dataset)**

## Section 1: Environment Setup

In [1]:
%%capture
# Install required packages
!pip install dask[dataframe] pyarrow xgboost lightgbm scikit-learn==1.4.2 python-dateutil -q

In [2]:
# Core libraries
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import json
import warnings
from datetime import datetime, timedelta
from dateutil import parser as date_parser
import logging
import gc
import os
import glob

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error
)
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import lightgbm as lgb
import joblib

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Environment setup complete.")
print(f"Python Libraries Loaded:")
print(f"  - pandas: {pd.__version__}")
print(f"  - numpy: {np.__version__}")
print(f"  - dask: {dd.__version__ if hasattr(dd, '__version__') else 'installed'}")
import sklearn
print(f"  - scikit-learn: {sklearn.__version__}")
print(f"  - xgboost: {xgb.__version__}")
print(f"  - lightgbm: {lgb.__version__}")

Environment setup complete.
Python Libraries Loaded:
  - pandas: 2.2.2
  - numpy: 2.0.2
  - dask: installed
  - scikit-learn: 1.4.2
  - xgboost: 3.1.3
  - lightgbm: 4.6.0


## Section 2: Configuration

In [3]:
# ══════════════════════════════════════════════════════════════════════════════
#                           CONFIGURATION - AUTO PATH DETECTION
# ══════════════════════════════════════════════════════════════════════════════

import os
import glob
from pathlib import Path

print("="*80)
print("PATH DETECTION")
print("="*80)

# Print environment info
print(f"Current working directory: {os.getcwd()}")
print(f"Home directory: {os.path.expanduser('~')}")

# Try to find the data directory by searching common locations
possible_paths = [
    # Relative paths (if notebook is in notebooks/)
    '../data/complete_data',
    'data/complete_data',
    # Absolute paths
    '/home/sj/Downloads/dynamic-pricing/data/complete_data',
    os.path.expanduser('~/Downloads/dynamic-pricing/data/complete_data'),
    # From current working directory
    os.path.join(os.getcwd(), 'data', 'complete_data'),
    os.path.join(os.getcwd(), '..', 'data', 'complete_data'),
    # Check parent directories
    os.path.join(Path(os.getcwd()).parent, 'data', 'complete_data'),
]

DATA_DIR = None
for path in possible_paths:
    abs_path = os.path.abspath(path)
    exists = os.path.exists(abs_path)
    if exists:
        parquet_count = len(glob.glob(os.path.join(abs_path, '*.parquet')))
        print(f"  ✓ {abs_path} - {parquet_count} parquet files")
        if parquet_count > 0 and DATA_DIR is None:
            DATA_DIR = abs_path
    else:
        print(f"  ✗ {abs_path} - does not exist")

if DATA_DIR is None:
    print("\n⚠️  Could not auto-detect data directory!")
    print("Please set DATA_DIR manually below:")
    print("DATA_DIR = '/your/actual/path/to/complete_data'")
    # Try to list what's in current directory
    print(f"\nContents of {os.getcwd()}:")
    for item in os.listdir(os.getcwd())[:20]:
        print(f"  {item}")
else:
    print(f"\n✓ Using data directory: {DATA_DIR}")

# Save directory (relative to data dir)
if DATA_DIR:
    SAVE_DIR = os.path.abspath(os.path.join(DATA_DIR, '..', '..', 'models', 'saved_runs'))
else:
    SAVE_DIR = './models/saved_runs'

CONFIG = {
    'data_dir': DATA_DIR,
    'save_dir': SAVE_DIR,
    'sample_fraction': 1.0,
    'min_price': 100,
    'max_price': 10000,
    'outlier_method': 'iqr_1.5',
    'test_size': 0.15,
    'shuffle': True,
    'models_to_train': ['LightGBM', 'XGBoost'],
    'random_state': 42
}

if DATA_DIR:
    os.makedirs(CONFIG['save_dir'], exist_ok=True)
    print(f"\nConfiguration ready. Found {len(glob.glob(os.path.join(DATA_DIR, '*.parquet')))} parquet files.")

PATH DETECTION
Current working directory: /content
Home directory: /root
  ✗ /data/complete_data - does not exist
  ✗ /content/data/complete_data - does not exist
  ✗ /home/sj/Downloads/dynamic-pricing/data/complete_data - does not exist
  ✗ /root/Downloads/dynamic-pricing/data/complete_data - does not exist
  ✗ /content/data/complete_data - does not exist
  ✗ /data/complete_data - does not exist
  ✗ /data/complete_data - does not exist

⚠️  Could not auto-detect data directory!
Please set DATA_DIR manually below:
DATA_DIR = '/your/actual/path/to/complete_data'

Contents of /content:
  .config
  sample_data


## Section 3: Data Loading with Dask

In [4]:
# ══════════════════════════════════════════════════════════════════════════════
#                    LOAD ALL PARQUET FILES WITH DASK
# ══════════════════════════════════════════════════════════════════════════════

print("Loading data with Dask...")
print("="*80)

# Use CONFIG from previous cell
data_dir = CONFIG['data_dir']
print(f"Using data directory: {data_dir}")

# Discover all parquet files
parquet_pattern = os.path.join(data_dir, '*.parquet')
print(f"Pattern: {parquet_pattern}")

all_files = sorted(glob.glob(parquet_pattern))
print(f"Found {len(all_files)} parquet files")

if not all_files:
    # Debug: try listing directory directly
    print("\nDEBUG: Listing directory contents...")
    if os.path.exists(data_dir):
        contents = os.listdir(data_dir)
        print(f"Directory contains {len(contents)} items")
        print(f"First 5: {contents[:5]}")
    else:
        print(f"Directory does not exist: {data_dir}")
    raise FileNotFoundError(f"No parquet files found in {data_dir}")

print(f"\nFirst 3 files: {all_files[:3]}")

# Load with Dask (lazy loading - doesn't load into memory yet)
ddf = dd.read_parquet(parquet_pattern, engine='pyarrow')

print(f"\nDask DataFrame created (lazy):")
print(f"  Partitions: {ddf.npartitions}")
print(f"  Columns: {len(ddf.columns)}")
print(f"\nColumn names:")
print(ddf.columns.tolist())

Loading data with Dask...
Using data directory: None


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    COMPUTE TOTAL ROWS (for progress tracking)
# ══════════════════════════════════════════════════════════════════════════════

print("Computing total row count...")
with ProgressBar():
    total_rows = len(ddf)
    
print(f"\nTotal rows in dataset: {total_rows:,}")

## Section 4: Data Parsing & Transformation

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    PRICE EXTRACTION FUNCTION
# ══════════════════════════════════════════════════════════════════════════════

def extract_price(df):
    """
    Extract price with priority:
    1. seat_salePrice (if > 0)
    2. seat_purchasePrice (if > 0)
    3. seat_purchasePrice + seat_tax
    """
    price = df['seat_salePrice'].fillna(0).astype(float)
    
    # Where sale price is invalid, use purchase price
    use_purchase = price <= 0
    price = price.where(~use_purchase, df['seat_purchasePrice'].fillna(0))
    
    # Where still invalid, add tax
    use_tax = (price <= 0) & df['seat_purchasePrice'].notna() & df['seat_tax'].notna()
    price = price.where(~use_tax, df['seat_purchasePrice'].fillna(0) + df['seat_tax'].fillna(0))
    
    return price

print("Price extraction function defined.")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    PARSE AND TRANSFORM DATA
# ══════════════════════════════════════════════════════════════════════════════

print("Parsing and transforming data...")
print("="*80)

# Apply transformations using Dask
# Note: We'll compute into pandas at the end for ML training

def parse_partition(pdf):
    """
    Parse a single partition (pandas DataFrame).
    This function will be applied to each Dask partition.
    """
    df = pdf.copy()
    
    # ─── Datetime Parsing ───────────────────────────────────────────────────
    for col, new_col in [('departureTime', 'departure_time'), 
                          ('arrivalTime', 'arrival_time'),
                          ('doj', 'journey_date')]:
        if col in df.columns:
            df[new_col] = pd.to_datetime(df[col], errors='coerce', utc=True)
    
    if 'scraped_time' in df.columns:
        df['scraped_time'] = pd.to_datetime(df['scraped_time'], errors='coerce', utc=True)
    
    # ─── Numeric Normalization ──────────────────────────────────────────────
    if 'availableSeats' in df.columns:
        df['available_seats'] = pd.to_numeric(df['availableSeats'], errors='coerce').fillna(0).astype(int)
    if 'avlWindowSeats' in df.columns:
        df['window_seats'] = pd.to_numeric(df['avlWindowSeats'], errors='coerce').fillna(0).astype(int)
    
    # ─── Boolean Flags ──────────────────────────────────────────────────────
    if 'is_AC' in df.columns:
        df['is_AC'] = df['is_AC'].astype(bool)
    
    # Seat-level flags
    for col in ['seat_is_upper', 'seat_is_ladiesSeat', 'seat_is_horizontal', 
                'seat_is_seater', 'seat_is_available']:
        if col in df.columns:
            df[col] = df[col].fillna(False).astype(bool)
        else:
            df[col] = False
    
    # Rename seat_is_ladiesSeat for consistency
    if 'seat_is_ladiesSeat' in df.columns:
        df['seat_is_ladies'] = df['seat_is_ladiesSeat']
    
    # ─── Price Extraction ───────────────────────────────────────────────────
    df['price'] = extract_price(df)
    
    # ─── Column Renaming ────────────────────────────────────────────────────
    rename_map = {
        'id': 'bus_id',
        'providerId': 'provider_id',
        'travels': 'operator_name',
        'busType': 'bus_type',
        'name': 'seat_name'
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
    
    return df

# Apply parsing to all partitions
print("Applying transformations to all partitions...")
ddf_parsed = ddf.map_partitions(parse_partition)

print("Transformation pipeline defined (lazy).")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    SELECT FINAL COLUMNS & COMPUTE
# ══════════════════════════════════════════════════════════════════════════════

# Define columns we need
final_columns = [
    'bus_id', 'provider_id', 'scraped_time', 'journey_date',
    'departure_time', 'arrival_time', 'source_collection',
    'bus_type', 'is_AC', 'available_seats', 'window_seats',
    'operator_name', 'seat_name', 'seat_is_upper', 'seat_is_ladies',
    'seat_is_horizontal', 'seat_is_seater', 'seat_is_available', 'price'
]

print("Computing full dataset into memory...")
print("(This may take a few minutes for large datasets)")
print("="*80)

with ProgressBar():
    # Compute to pandas DataFrame
    df_raw = ddf_parsed.compute()

# Select only columns that exist
existing_cols = [c for c in final_columns if c in df_raw.columns]
df_raw = df_raw[existing_cols].copy()

print(f"\nData loaded successfully!")
print(f"  Shape: {df_raw.shape}")
print(f"  Memory: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  Columns: {df_raw.columns.tolist()}")

# Clean up Dask objects
del ddf, ddf_parsed
gc.collect()

## Section 5: Data Preprocessing

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    PREPROCESSING CLASS
# ══════════════════════════════════════════════════════════════════════════════

class BusSeatPreprocessor:
    def __init__(self, min_price=100, max_price=10000, outlier_method='iqr_1.5'):
        self.min_price = min_price
        self.max_price = max_price
        self.outlier_method = outlier_method
        self.stats = {}
    
    def remove_duplicates(self, df):
        n = len(df)
        subset_cols = ['bus_id', 'seat_name', 'journey_date', 'scraped_time']
        subset_cols = [c for c in subset_cols if c in df.columns]
        if subset_cols:
            df = df.drop_duplicates(subset=subset_cols)
        self.stats['duplicates_removed'] = n - len(df)
        return df
    
    def clean_invalid(self, df):
        n = len(df)
        
        # Valid seats
        if 'available_seats' in df.columns and 'window_seats' in df.columns:
            df = df[(df['available_seats'] >= 0) & (df['window_seats'] >= 0)]
            df = df[df['window_seats'] <= df['available_seats']]
        
        # Valid price range
        df = df[df['price'].between(self.min_price, self.max_price)]
        
        self.stats['invalid_removed'] = n - len(df)
        return df
    
    def remove_outliers(self, df):
        n = len(df)
        
        if self.outlier_method == 'iqr_1.5':
            Q1, Q3 = df['price'].quantile([0.25, 0.75])
            IQR = Q3 - Q1
            lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
            df = df[(df['price'] >= lower) & (df['price'] <= upper)]
        elif self.outlier_method == 'iqr_3':
            Q1, Q3 = df['price'].quantile([0.25, 0.75])
            IQR = Q3 - Q1
            lower, upper = Q1 - 3 * IQR, Q3 + 3 * IQR
            df = df[(df['price'] >= lower) & (df['price'] <= upper)]
        
        self.stats['outliers_removed'] = n - len(df)
        return df
    
    def add_time_features(self, df):
        """Add time-based features."""
        
        # Hours to departure
        if all(c in df.columns for c in ['departure_time', 'scraped_time']):
            df['hours_to_departure'] = (
                (df['departure_time'] - df['scraped_time'])
                .dt.total_seconds() / 3600
            ).clip(lower=0)
        
        # Trip duration
        if all(c in df.columns for c in ['arrival_time', 'departure_time']):
            df['duration_hours'] = (
                (df['arrival_time'] - df['departure_time'])
                .dt.total_seconds() / 3600
            ).clip(lower=0)
        
        # Days to journey
        if all(c in df.columns for c in ['journey_date', 'scraped_time']):
            df['days_to_journey'] = (
                (df['journey_date'].dt.floor('D') - df['scraped_time'].dt.floor('D'))
                .dt.days
            ).clip(lower=0)
        
        # Time extractions
        if 'scraped_time' in df.columns:
            df['scrape_hour'] = df['scraped_time'].dt.hour
        
        if 'journey_date' in df.columns:
            df['journey_weekday'] = df['journey_date'].dt.weekday
        
        if 'departure_time' in df.columns:
            df['departure_hour'] = df['departure_time'].dt.hour
        
        # Log price (target)
        df['price_log'] = np.log1p(df['price'])
        
        return df
    
    def run(self, df):
        n_start = len(df)
        print(f"Starting preprocessing with {n_start:,} rows")
        
        df = self.remove_duplicates(df)
        df = self.clean_invalid(df)
        df = self.remove_outliers(df)
        df = self.add_time_features(df)
        
        n_end = len(df)
        self.stats['retention_pct'] = round(n_end / n_start * 100, 1)
        
        print(f"\nPreprocessing Summary:")
        for k, v in self.stats.items():
            print(f"  {k.replace('_', ' ').title():<25}: {v}")
        print(f"  Final rows: {n_end:,}")
        
        return df

print("Preprocessor class defined.")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    EXECUTE PREPROCESSING
# ══════════════════════════════════════════════════════════════════════════════

preprocessor = BusSeatPreprocessor(
    min_price=CONFIG['min_price'],
    max_price=CONFIG['max_price'],
    outlier_method=CONFIG['outlier_method']
)

df_clean = preprocessor.run(df_raw.copy())

print(f"\nAfter preprocessing: {df_clean.shape}")
display(df_clean.head(3))

## Section 6: Feature Engineering

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    FEATURE ENGINEERING CLASS
# ══════════════════════════════════════════════════════════════════════════════

class BusPriceFeatureEngineer:
    def __init__(self):
        self.new_features = []
    
    def create_features(self, df):
        df = df.copy()
        original_cols = set(df.columns)
        
        print(f"Starting Feature Engineering with shape: {df.shape}")
        
        # ─── Temporal Patterns ──────────────────────────────────────────────
        if 'journey_weekday' in df.columns:
            df['journey_is_weekend'] = df['journey_weekday'].isin([5, 6]).astype(int)
        
        if 'departure_hour' in df.columns:
            # Night departure (8 PM - 5 AM)
            df['is_night_departure'] = ((df['departure_hour'] >= 20) | (df['departure_hour'] <= 5)).astype(int)
            # Peak hours (6-9 AM, 5-8 PM)
            df['is_peak_hour'] = (df['departure_hour'].isin([6,7,8,9,17,18,19,20])).astype(int)
        
        # ─── Booking Window ─────────────────────────────────────────────────
        if 'hours_to_departure' in df.columns:
            df['is_last_minute'] = (df['hours_to_departure'] <= 6).astype(int)
            df['is_advance_booking'] = (df['hours_to_departure'] >= 168).astype(int)  # 7+ days
        
        # ─── Demand & Scarcity Signals ──────────────────────────────────────
        if 'available_seats' in df.columns:
            df['low_availability'] = (df['available_seats'] <= 5).astype(int)
            df['very_low_availability'] = (df['available_seats'] <= 2).astype(int)
            df['seats_sold_ratio'] = (1 - (df['available_seats'] / 50).clip(upper=1))
        
        # ─── Seat Characteristics ───────────────────────────────────────────
        if 'seat_is_upper' in df.columns:
            df['is_lower_berth'] = (~df['seat_is_upper']).astype(int)
        
        if 'window_seats' in df.columns and 'seat_is_upper' in df.columns:
            df['is_premium_seat'] = ((~df['seat_is_upper']) & (df['window_seats'] > 0)).astype(int)
        
        # ─── Bus Type Features ──────────────────────────────────────────────
        if 'bus_type' in df.columns:
            bus_type_lower = df['bus_type'].str.lower().fillna('')
            df['is_volvo'] = bus_type_lower.str.contains('volvo').astype(int)
            df['is_sleeper'] = bus_type_lower.str.contains('sleeper').astype(int)
            df['is_seater'] = bus_type_lower.str.contains('seater').astype(int)
            df['is_multi_axle'] = bus_type_lower.str.contains('multi|axle').astype(int)
        
        self.new_features = sorted(set(df.columns) - original_cols)
        print(f"Feature Engineering complete. Shape: {df.shape}")
        print(f"New features added: {self.new_features}")
        
        return df

print("Feature Engineer class defined.")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    EXECUTE FEATURE ENGINEERING
# ══════════════════════════════════════════════════════════════════════════════

fe = BusPriceFeatureEngineer()
df_features = fe.create_features(df_clean.copy())

print(f"\nAfter Feature Engineering: {df_features.shape}")
display(df_features.head(3))

## Section 7: Categorical Encoding

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    CATEGORICAL ENCODING
# ══════════════════════════════════════════════════════════════════════════════

class CategoricalEncoder:
    def __init__(self):
        self.encoders = {}
        self.encoded_columns = []
    
    def encode(self, df):
        df = df.copy()
        print(f"Starting categorical encoding with shape: {df.shape}")
        
        # High cardinality columns to encode
        high_card_cols = ['operator_name', 'bus_type', 'source_collection', 'seat_name']
        
        for col in high_card_cols:
            if col in df.columns:
                le = LabelEncoder()
                encoded_col = f'{col}_le'
                
                # Convert to string and encode
                df[encoded_col] = le.fit_transform(df[col].astype(str))
                
                self.encoders[col] = le
                self.encoded_columns.append(encoded_col)
                print(f"  Encoded: {col} → {encoded_col} ({len(le.classes_)} classes)")
        
        print(f"\nEncoding complete. Shape: {df.shape}")
        return df

# Execute encoding
encoder = CategoricalEncoder()
df_encoded = encoder.encode(df_features.copy())

print(f"\nEncoded columns: {encoder.encoded_columns}")
display(df_encoded.head(3))

## Section 8: Train-Test Split (GLOBAL SHUFFLE)

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    PREPARE FEATURES AND TARGET
# ══════════════════════════════════════════════════════════════════════════════

print("Preparing features and target...")
print("="*80)

# Define target
target_col = 'price_log'

# Columns to exclude from features
exclude_cols = [
    'price', 'price_log', 'price_capped',  # Target related
    'departure_time', 'arrival_time', 'journey_date', 'scraped_time',  # Datetime objects
    'bus_id', 'provider_id', 'seat_name',  # IDs (use encoded versions)
    'operator_name', 'bus_type', 'source_collection',  # Categoricals (use encoded versions)
    'hours_to_departure_bin'  # Categorical bin
]

# Select numeric features only
numeric_cols = df_encoded.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [c for c in numeric_cols if c not in exclude_cols]

print(f"Target: {target_col}")
print(f"Features ({len(feature_cols)}): {feature_cols}")

# Create X and y
X = df_encoded[feature_cols].copy()
y = df_encoded[target_col].copy()

# Handle any remaining NaN/inf values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"NaN in X: {X.isna().sum().sum()}")
print(f"NaN in y: {y.isna().sum()}")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#         CRITICAL: GLOBAL SHUFFLE + TRAIN-TEST SPLIT
# ══════════════════════════════════════════════════════════════════════════════

print("Performing GLOBAL shuffle and train-test split...")
print("="*80)
print(f"Test size: {CONFIG['test_size']}")
print(f"Shuffle: {CONFIG['shuffle']}")
print(f"Random state: {CONFIG['random_state']}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=CONFIG['test_size'],
    random_state=CONFIG['random_state'],
    shuffle=CONFIG['shuffle']  # CRITICAL: Global shuffle!
)

print(f"\nSplit Results:")
print(f"  Training set: {X_train.shape[0]:,} samples ({100-CONFIG['test_size']*100:.0f}%)")
print(f"  Test set:     {X_test.shape[0]:,} samples ({CONFIG['test_size']*100:.0f}%)")
print(f"  Features:     {X_train.shape[1]}")

# Verify distributions are similar (important check!)
print(f"\nTarget Distribution Check:")
print(f"  Train mean: {y_train.mean():.4f}, std: {y_train.std():.4f}")
print(f"  Test mean:  {y_test.mean():.4f}, std: {y_test.std():.4f}")

## Section 9: Model Training

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    MODEL TRAINER CLASS
# ══════════════════════════════════════════════════════════════════════════════

class ModelTrainer:
    def __init__(self, X_train, X_test, y_train, y_test, random_state=42):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.random_state = random_state
        
        self.models = {}
        self.scores = {}
        self.best_model_name = None
        self.best_model = None
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def _evaluate_model(self, model, name):
        """Evaluate model and store metrics."""
        y_pred_log = model.predict(self.X_test)
        
        # Convert from log scale to original scale for INR metrics
        y_pred_inr = np.expm1(y_pred_log)
        y_test_inr = np.expm1(self.y_test)
        
        self.scores[name] = {
            'rmse_log': mean_squared_error(self.y_test, y_pred_log, squared=False),
            'mae_log': mean_absolute_error(self.y_test, y_pred_log),
            'r2_log': r2_score(self.y_test, y_pred_log),
            'rmse_inr': mean_squared_error(y_test_inr, y_pred_inr, squared=False),
            'mae_inr': mean_absolute_error(y_test_inr, y_pred_inr),
            'mape_pct': mean_absolute_percentage_error(y_test_inr, y_pred_inr) * 100
        }
        
        self.logger.info(
            f"{name} Test Metrics → "
            f"RMSE(₹): {self.scores[name]['rmse_inr']:.0f} | "
            f"MAPE: {self.scores[name]['mape_pct']:.2f}% | "
            f"R²: {self.scores[name]['r2_log']:.4f}"
        )
    
    def train_lightgbm(self):
        """Train LightGBM model."""
        self.logger.info("Training LightGBM...")
        
        model = lgb.LGBMRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=8,
            num_leaves=64,
            min_child_samples=20,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.1,
            reg_lambda=0.1,
            random_state=self.random_state,
            n_jobs=-1,
            verbosity=-1
        )
        
        model.fit(
            self.X_train, self.y_train,
            eval_set=[(self.X_test, self.y_test)],
            callbacks=[lgb.early_stopping(50, verbose=False)]
        )
        
        self.models['LightGBM'] = model
        self._evaluate_model(model, 'LightGBM')
    
    def train_xgboost(self):
        """Train XGBoost model."""
        self.logger.info("Training XGBoost...")
        
        model = xgb.XGBRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=8,
            min_child_weight=5,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.1,
            reg_lambda=0.1,
            random_state=self.random_state,
            n_jobs=-1,
            verbosity=0
        )
        
        model.fit(
            self.X_train, self.y_train,
            eval_set=[(self.X_test, self.y_test)],
            verbose=False
        )
        
        self.models['XGBoost'] = model
        self._evaluate_model(model, 'XGBoost')
    
    def train_all(self, models_to_train=None):
        """Train all specified models."""
        if models_to_train is None:
            models_to_train = ['LightGBM', 'XGBoost']
        
        print("="*80)
        print("MODEL TRAINING")
        print("="*80)
        
        if 'LightGBM' in models_to_train:
            self.train_lightgbm()
        
        if 'XGBoost' in models_to_train:
            self.train_xgboost()
        
        # Find best model (lowest RMSE in INR)
        if self.scores:
            scores_df = pd.DataFrame(self.scores).T
            self.best_model_name = scores_df['rmse_inr'].idxmin()
            self.best_model = self.models[self.best_model_name]
            
            print("\n" + "="*80)
            print("MODEL COMPARISON")
            print("="*80)
            display(scores_df.round(4))
            
            print(f"\nBEST MODEL: {self.best_model_name}")
            print(f"  → RMSE (₹): {self.scores[self.best_model_name]['rmse_inr']:.0f}")
            print(f"  → MAE (₹):  {self.scores[self.best_model_name]['mae_inr']:.0f}")
            print(f"  → MAPE:     {self.scores[self.best_model_name]['mape_pct']:.2f}%")
            print(f"  → R² (log): {self.scores[self.best_model_name]['r2_log']:.4f}")

print("ModelTrainer class defined.")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    TRAIN MODELS
# ══════════════════════════════════════════════════════════════════════════════

trainer = ModelTrainer(
    X_train, X_test, y_train, y_test,
    random_state=CONFIG['random_state']
)

trainer.train_all(models_to_train=CONFIG['models_to_train'])

# Export for later use
best_model_name = trainer.best_model_name
best_model = trainer.best_model

print(f"\nExported: best_model_name = '{best_model_name}'")

## Section 10: Model Evaluation & Visualization

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    DETAILED EVALUATION
# ══════════════════════════════════════════════════════════════════════════════

print("="*80)
print(f"DETAILED EVALUATION: {best_model_name}")
print("="*80)

# Get predictions
y_pred_log = best_model.predict(X_test)
y_pred_inr = np.expm1(y_pred_log)
y_test_inr = np.expm1(y_test)

# Calculate metrics
metrics = {
    'R² (log scale)': r2_score(y_test, y_pred_log),
    'RMSE (₹)': mean_squared_error(y_test_inr, y_pred_inr, squared=False),
    'MAE (₹)': mean_absolute_error(y_test_inr, y_pred_inr),
    'MAPE (%)': mean_absolute_percentage_error(y_test_inr, y_pred_inr) * 100
}

print("\nFinal Model Metrics:")
for name, value in metrics.items():
    print(f"  {name}: {value:.4f}")

# R² interpretation
r2 = metrics['R² (log scale)']
if r2 >= 0.8:
    print(f"\n✅ Excellent R² of {r2:.4f}! Model explains {r2*100:.1f}% of variance.")
elif r2 >= 0.7:
    print(f"\n✅ Good R² of {r2:.4f}! Model explains {r2*100:.1f}% of variance.")
elif r2 >= 0.5:
    print(f"\n⚠️ Moderate R² of {r2:.4f}. Model explains {r2*100:.1f}% of variance.")
else:
    print(f"\n❌ Low R² of {r2:.4f}. Model needs improvement.")

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    VISUALIZATION
# ══════════════════════════════════════════════════════════════════════════════

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Actual vs Predicted (log scale)
ax = axes[0, 0]
ax.scatter(y_test, y_pred_log, alpha=0.3, s=5)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax.set_xlabel('Actual (log price)')
ax.set_ylabel('Predicted (log price)')
ax.set_title(f'{best_model_name} - Actual vs Predicted (log scale)\nR² = {r2:.4f}')
ax.grid(True, alpha=0.3)

# 2. Actual vs Predicted (INR scale)
ax = axes[0, 1]
ax.scatter(y_test_inr, y_pred_inr, alpha=0.3, s=5)
ax.plot([y_test_inr.min(), y_test_inr.max()], [y_test_inr.min(), y_test_inr.max()], 'r--', lw=2)
ax.set_xlabel('Actual Price (₹)')
ax.set_ylabel('Predicted Price (₹)')
ax.set_title(f'{best_model_name} - Actual vs Predicted (₹)\nRMSE = ₹{metrics["RMSE (₹)"]:.0f}')
ax.grid(True, alpha=0.3)

# 3. Residual Plot
ax = axes[1, 0]
residuals = y_test_inr - y_pred_inr
ax.scatter(y_pred_inr, residuals, alpha=0.3, s=5)
ax.axhline(y=0, color='r', linestyle='--', lw=2)
ax.set_xlabel('Predicted Price (₹)')
ax.set_ylabel('Residual (₹)')
ax.set_title(f'{best_model_name} - Residual Plot')
ax.grid(True, alpha=0.3)

# 4. Residual Distribution
ax = axes[1, 1]
ax.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='r', linestyle='--', lw=2)
ax.set_xlabel('Residual (₹)')
ax.set_ylabel('Frequency')
ax.set_title(f'{best_model_name} - Residual Distribution\nMean: ₹{residuals.mean():.0f}, Std: ₹{residuals.std():.0f}')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Section 11: Feature Importance

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    FEATURE IMPORTANCE ANALYSIS
# ══════════════════════════════════════════════════════════════════════════════

print("="*80)
print(f"FEATURE IMPORTANCE - {best_model_name}")
print("="*80)

# Get feature importance
if best_model_name == 'LightGBM':
    importance = best_model.feature_importances_
elif best_model_name == 'XGBoost':
    importance = best_model.feature_importances_

# Create DataFrame
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': importance
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features:")
print(importance_df.head(20).to_string(index=False))

# Plot
top_n = 15
plt.figure(figsize=(10, 8))
top_features = importance_df.head(top_n)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance')
plt.title(f'Top {top_n} Feature Importances - {best_model_name}')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Section 12: Model Persistence

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    SAVE MODEL AND ARTIFACTS
# ══════════════════════════════════════════════════════════════════════════════

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

print("="*80)
print("MODEL PERSISTENCE")
print("="*80)

# 1. Save best model
model_filename = os.path.join(CONFIG['save_dir'], f'bus_price_model_{best_model_name}_DASK_{timestamp}.pkl')
joblib.dump(best_model, model_filename)
print(f"Saved model: {model_filename}")

# 2. Save categorical encoders
encoder_filename = os.path.join(CONFIG['save_dir'], f'categorical_encoder_DASK_{timestamp}.pkl')
joblib.dump(encoder.encoders, encoder_filename)
print(f"Saved encoders: {encoder_filename}")

# 3. Save feature names
features_filename = os.path.join(CONFIG['save_dir'], f'feature_names_DASK_{timestamp}.json')
with open(features_filename, 'w') as f:
    json.dump(feature_cols, f, indent=2)
print(f"Saved {len(feature_cols)} feature names: {features_filename}")

# 4. Save feature engineer
fe_filename = os.path.join(CONFIG['save_dir'], f'feature_engineer_DASK_{timestamp}.pkl')
joblib.dump(fe, fe_filename)
print(f"Saved feature engineer: {fe_filename}")

# 5. Save metrics
metrics_filename = os.path.join(CONFIG['save_dir'], f'model_metrics_DASK_{timestamp}.json')
metrics_data = {
    'model_name': best_model_name,
    'timestamp': timestamp,
    'approach': 'DASK_FULL_SHUFFLE',
    'metrics': {
        'r2_log': float(metrics['R² (log scale)']),
        'rmse_inr': float(metrics['RMSE (₹)']),
        'mae_inr': float(metrics['MAE (₹)']),
        'mape_pct': float(metrics['MAPE (%)'])
    },
    'training_samples': int(len(X_train)),
    'testing_samples': int(len(X_test)),
    'num_features': len(feature_cols),
    'config': CONFIG
}

with open(metrics_filename, 'w') as f:
    json.dump(metrics_data, f, indent=2, default=str)
print(f"Saved metrics: {metrics_filename}")

print("\n" + "="*80)
print("✅ MODEL PERSISTENCE COMPLETE!")
print("="*80)
print(f"\nFiles saved in: {os.path.abspath(CONFIG['save_dir'])}")

## Section 13: Summary

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    FINAL SUMMARY
# ══════════════════════════════════════════════════════════════════════════════

print("\n" + "═"*80)
print("                    TRAINING COMPLETE - SUMMARY")
print("═"*80)

print(f"\n📊 Dataset:")
print(f"   Total samples processed: {len(df_encoded):,}")
print(f"   Training samples: {len(X_train):,}")
print(f"   Test samples: {len(X_test):,}")
print(f"   Features used: {len(feature_cols)}")

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   R² (log scale):  {metrics['R² (log scale)']:.4f}")
print(f"   RMSE (₹):        {metrics['RMSE (₹)']:.0f}")
print(f"   MAE (₹):         {metrics['MAE (₹)']:.0f}")
print(f"   MAPE (%):        {metrics['MAPE (%)']:.2f}%")

print(f"\n💾 Saved Artifacts:")
print(f"   Model: {model_filename}")
print(f"   Encoders: {encoder_filename}")
print(f"   Features: {features_filename}")
print(f"   Metrics: {metrics_filename}")

print("\n" + "═"*80)
print("                    ✅ READY FOR PRODUCTION")
print("═"*80)

## Section 14: Sample Prediction (Demo)

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
#                    SAMPLE PREDICTION
# ══════════════════════════════════════════════════════════════════════════════

print("Sample Predictions on Test Data:")
print("="*80)

# Get some sample predictions
sample_idx = np.random.choice(len(X_test), size=10, replace=False)
X_sample = X_test.iloc[sample_idx]
y_sample_actual = np.expm1(y_test.iloc[sample_idx])
y_sample_pred = np.expm1(best_model.predict(X_sample))

# Create comparison DataFrame
comparison = pd.DataFrame({
    'Actual (₹)': y_sample_actual.values,
    'Predicted (₹)': y_sample_pred,
    'Difference (₹)': y_sample_actual.values - y_sample_pred,
    'Error (%)': abs(y_sample_actual.values - y_sample_pred) / y_sample_actual.values * 100
})

print(comparison.round(2).to_string(index=False))

print(f"\nAverage absolute error: ₹{abs(comparison['Difference (₹)']).mean():.0f}")
print(f"Average percentage error: {comparison['Error (%)'].mean():.1f}%")